In [ ]:
import numpy as np
import torch
import os
import pickle
from model import GPT, GPTConfig
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
ALPHABET = [chr(i) for i in range(ord('a'), ord('z') + 1)]
SEP_BAR, SEP_Q = '|', '?'
batch_size = 64
block_size = 2048
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
device_type = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# load model checkpoint
mono_alph_checkpoint_file = 'out/gpt2-BS-64/gpt2-BS-64_786M_ckpt.pt'
vig_4_checkpoint_file = 'out/gpt2-vignere-scheme/gpt2-vignere-scheme_655M_ckpt.pt'
vig_32_checkpoint_file = 'out/gpt2-vignere-scheme-key-L-32-Block3072/gpt2-vignere-scheme-key-L-32-Block3072_3932M_ckpt.pt'
vig_rand_checkpoint_file = 'out/gpt2-vignere-scheme-key-L4thru32-Block3072-bs64/gpt2-vignere-scheme-key-L4thru32-Block3072-bs64_983M_ckpt.pt'
checkpoint = torch.load(vig_rand_checkpoint_file, map_location=device)
checkpoint_model_args = checkpoint['model_args']
n_layer=12
n_head=8
n_embd=256
bias=False
dropout=0.0
model_args = dict(n_layer=n_layer, n_head=n_head, n_embd=n_embd, block_size=block_size,
                  bias=bias, vocab_size=None, dropout=dropout) # start with model_args from command line
# force these config attributes to be equal otherwise we can't even resume training
# the rest of the attributes (e.g. dropout) can stay as desired from command line
for k in ['n_layer', 'n_head', 'n_embd', 'block_size', 'bias', 'vocab_size']:
    model_args[k] = checkpoint_model_args[k]
# create the model
gptconf = GPTConfig(**model_args)
model = GPT(gptconf)
state_dict = checkpoint['model']
# fix the keys of the state dictionary :(
# honestly no idea how checkpoints sometimes get this prefix, have to debug more
unwanted_prefix = '_orig_mod.'
for k,v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
model.load_state_dict(state_dict)
state_dict = None
iter_num = checkpoint['iter_num']
best_val_loss = checkpoint['best_val_loss']
total_tokens = checkpoint['total_tokens']
model.eval()

In [ ]:
dataset = 'openwebtext'
# poor man's data loader
data_dir = os.path.join('data', dataset)
with open(os.path.join(data_dir, 'meta.pkl'), 'rb') as f:
    meta = pickle.load(f)
stoi, itos = meta['stoi'], meta['itos']
vocab_size = meta['vocab_size'] 
print(f"Using vocab size of {vocab_size} (a-z + separators)")

# ---------------- helper: random mono‑alphabetic key --------
alpha_ids = np.array([stoi[c] for c in ALPHABET], dtype=np.uint8)
def random_key():
    perm = np.random.permutation(26)
    enc  = {alpha_ids[i]: alpha_ids[perm[i]] for i in range(26)}   # plain→cipher
    dec  = {v: k for k, v in enc.items()}                          # cipher→plain
    return enc, dec

def get_batch(split):
    mmap = np.memmap(os.path.join(data_dir, f'{split}.bin'),
                     dtype=np.uint8, mode='r')

    k_pairs   = 1024                              # desired number of pairs
    known_k   = k_pairs - 1                       # last one is the query
    prompt_sz = 2 * k_pairs                       # 2048 tokens
    assert prompt_sz == block_size, "block_size must be 2*k_pairs"

    X = torch.full((batch_size, block_size - 1), stoi['|'],  dtype=torch.long)
    Y = torch.full((batch_size, block_size - 1), -1,          dtype=torch.long)

    for b in range(batch_size):
        # ----- 1. grab k plaintext letters from corpus -------------------
        start = np.random.randint(0, len(mmap) - k_pairs - 1)
        plain = mmap[start:start + k_pairs].copy()          # np.uint8, shape (k_pairs,)

        # ----- 2. fresh random key for this sample -----------------------
        enc, _ = random_key()

        # ----- 3. build prompt ------------------------------------------
        buf, tgt = [], []
        for i, p in enumerate(plain):
            c = enc[p]
            if i < known_k:                                 # give answer
                buf.extend([c, p])
                tgt.extend([p, -1])
            else:                                           # query pair
                buf.extend([c])
                tgt.extend([p])

        X[b] = torch.from_numpy(np.asarray(buf,  np.uint8))
        Y[b] = torch.from_numpy(np.asarray(tgt, np.int64))

    if device_type == 'cuda':
        X, Y = X.pin_memory().to(device, non_blocking=True), \
               Y.pin_memory().to(device, non_blocking=True)
    else:
        X, Y = X.to(device), Y.to(device)
    return X, Y

eval_iters = 50
dtype = 'bfloat16'
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = torch.amp.autocast(device_type=device_type, dtype=ptdtype)
model.to(device)
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            X.to(device)
            Y.to(device)
            with ctx:
                logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
l = estimate_loss()

In [ ]:
print(l['train'], l['val'])

In [ ]:
@torch.no_grad()
def visualize_samples(num_samples=5, split='val'):
    """
    Visualize samples from the dataset, showing the overall distribution of plaintext characters.
    
    Args:
        num_samples: Number of samples to process
        split: Dataset split to use ('train' or 'val')
    """
    model.eval()
    
    # For tracking the distribution of plaintext characters after query ciphertext
    query_cipher_to_plain = {}
    
    # For tracking model predictions vs true labels
    true_vs_pred = {}
    
    # Process samples to gather distribution data
    samples_processed = 0
    while samples_processed < num_samples:
        X, Y = get_batch(split)
        X = X.to(device)
        Y = Y.to(device)
        
        with ctx:
            logits, _ = model(X, Y)
        
        # For each batch item
        for i in range(X.shape[0]):
            # Find the last position in the sequence (where the model makes its prediction)
            seq_len = (X[i] != 0).sum().item()  # Count non-padding tokens
            if seq_len == 0:
                continue
                
            # The last position is where the prediction happens
            pred_pos = seq_len - 1
            
            # Get the true label
            true_label = Y[i, pred_pos].item()
            
            # Skip if true_label is -1 (padding)
            if true_label == -1:
                continue
            
            # Get model's prediction
            pred_label = logits[i, pred_pos].argmax(dim=-1).item()
            
            # Track true vs predicted
            true_char = itos[true_label]
            pred_char = itos[pred_label]
            
            if true_char not in true_vs_pred:
                true_vs_pred[true_char] = {}
            
            if pred_char not in true_vs_pred[true_char]:
                true_vs_pred[true_char][pred_char] = 0
                
            true_vs_pred[true_char][pred_char] += 1
            
            # Track the distribution of plaintext characters after query ciphertext
            # The last character in the sequence is the ciphertext we're querying about
            cipher_char = itos[X[i, pred_pos].item()]
            plain_char = itos[true_label]
            
            if cipher_char not in query_cipher_to_plain:
                query_cipher_to_plain[cipher_char] = {}
            
            if plain_char not in query_cipher_to_plain[cipher_char]:
                query_cipher_to_plain[cipher_char][plain_char] = 0
            
            query_cipher_to_plain[cipher_char][plain_char] += 1
            
            samples_processed += 1
            
            # Break if we have processed enough samples
            if samples_processed >= num_samples:
                break
    
    # Calculate the overall distribution of plaintext characters
    all_plain_counts = {}
    for cipher_dict in query_cipher_to_plain.values():
        for plain, count in cipher_dict.items():
            if plain not in all_plain_counts:
                all_plain_counts[plain] = 0
            all_plain_counts[plain] += count
    
    # Calculate total count for percentage calculation
    total_count = sum(all_plain_counts.values())
    
    # Calculate percentages
    all_plain_percentages = {plain: (count / total_count) * 100 for plain, count in all_plain_counts.items()}
    
    # First visualization: Overall distribution
    plt.figure(figsize=(12, 6))
    sorted_plains = sorted(all_plain_counts.keys())
    bars = plt.bar(sorted_plains, [all_plain_percentages[p] for p in sorted_plains])
    plt.title(f'Distribution of Final Plaintext Characters in Dataset (Total: {total_count} characters)', fontsize=18)
    plt.xlabel('Plaintext Character', fontsize=16)
    plt.ylabel('Percentage (%)', fontsize=16)
    plt.xticks(rotation=0, fontsize=14)
    plt.yticks(fontsize=14)
    
    # Set a higher y-axis limit to make room for percentage labels
    max_percentage = max(all_plain_percentages.values())
    plt.ylim(0, max_percentage * 1.2)  # Add 20% more space above the highest bar
    
    # Add percentage labels on top of each bar
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.5,
                f'{height:.1f}%', ha='center', va='bottom', fontsize=12)
    
    plt.tight_layout()
    plt.show()
    
    # Second visualization: Model predictions vs true labels
    plt.figure(figsize=(14, 10))
    
    # Get all unique characters
    all_chars = sorted(set(list(true_vs_pred.keys()) + 
                          [pred for true_dict in true_vs_pred.values() for pred in true_dict.keys()]))
    
    # Create confusion matrix
    confusion = np.zeros((len(all_chars), len(all_chars)))
    char_to_idx = {char: i for i, char in enumerate(all_chars)}
    
    for true_char, pred_dict in true_vs_pred.items():
        true_idx = char_to_idx[true_char]
        for pred_char, count in pred_dict.items():
            pred_idx = char_to_idx[pred_char]
            confusion[true_idx, pred_idx] = count
    
    # Normalize by row (true labels)
    row_sums = confusion.sum(axis=1, keepdims=True)
    confusion_norm = np.zeros_like(confusion)
    np.divide(confusion, row_sums, out=confusion_norm, where=row_sums!=0)
    
    # Plot heatmap
    plt.imshow(confusion_norm, cmap='Blues')
    cbar = plt.colorbar(label='Prediction Frequency')
    cbar.ax.tick_params(labelsize=14)
    cbar.set_label('Prediction Frequency', fontsize=16)
    plt.title('Model Predictions vs True Plaintext Characters', fontsize=18)
    plt.xlabel('Predicted Plaintext Character', fontsize=16)
    plt.ylabel('True Plaintext Character', fontsize=16)
    
    # Set ticks
    plt.xticks(np.arange(len(all_chars)), all_chars, fontsize=14)
    plt.yticks(np.arange(len(all_chars)), all_chars, fontsize=14)
    
    # Rotate x-axis labels
    plt.xticks(rotation=45)
    
    # Add text annotations
    for i in range(len(all_chars)):
        for j in range(len(all_chars)):
            if confusion_norm[i, j] > 0:
                text_color = 'white' if confusion_norm[i, j] > 0.5 else 'black'
                plt.text(j, i, f'{confusion_norm[i, j]:.2f}', 
                        ha='center', va='center', color=text_color,
                        fontsize=12)
    
    plt.tight_layout()
    plt.show()

# Visualize the overall distribution from the validation set
visualize_samples(num_samples=10000, split='val')


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Example data — replace with your actual values
learning_rates = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 3e0, 1e1]
validation_scores = [1.7007, 0.3562, 0.0850, 0.0075, 0.0030, 0.0033, 0.2328, 0.3596]

# Prepare heatmap data: 1 row, many columns
data = np.array([validation_scores])

# Convert LR to string for display
lr_labels = [f"{lr:.0e}" for lr in learning_rates]

# Set up the plot with wider figure
plt.figure(figsize=(16, 3))
ax = sns.heatmap(
    data,
    annot=True,
    fmt=".4f",
    cmap="viridis",
    xticklabels=False,  # Disable default x-tick labels
    yticklabels=["Val Loss"],
    cbar_kws={"label": "Validation Loss"},
    annot_kws={"fontsize": 18, "fontfamily": "monospace"}
)

# Manually set centered x-tick labels
ax.set_xticks(np.arange(len(lr_labels)) + 0.5)  # Centered ticks
ax.set_xticklabels(lr_labels, fontsize=18, rotation=0)

# Label and title with larger font sizes
ax.set_xlabel("Learning Rate", fontsize=20)
ax.set_ylabel("")
ax.set_yticklabels(ax.get_yticklabels(), fontsize=18)

# Increase font size of colorbar
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=18)
cbar.set_label("Val Loss", fontsize=20)

plt.tight_layout()
plt.show()

In [ ]:
import re
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

def extract_learning_rate(log_file_path):
    # Extract learning rate from filename (e.g., 'lr-1e-3.log' -> '1e-3')
    match = re.search(r'lr-([0-9e\-]+)', log_file_path)
    if match:
        return match.group(1)
    return "unknown"

def parse_log_file(log_file_path):
    with open(log_file_path, 'r') as f:
        log_data = f.read()
    
    # Extract learning rate from filename
    lr = extract_learning_rate(log_file_path)
    
    # Pattern to extract training loss from "iter X: loss Y"
    train_loss_pattern = re.compile(r"iter (\d+): loss ([\d.]+)")
    train_iters = []
    train_losses = []

    for match in train_loss_pattern.findall(log_data):
        iter_num = int(match[0])
        loss_val = float(match[1])
        train_iters.append(iter_num * 256)
        train_losses.append(loss_val)

    # Pattern to extract validation loss from "step X: train loss A, val loss B"
    val_loss_pattern = re.compile(r"step (\d+): train loss [\d.]+, val loss ([\d.]+)")
    val_steps = []
    val_losses = []

    for match in val_loss_pattern.findall(log_data):
        step_num = int(match[0])
        val_loss_val = float(match[1])
        val_steps.append(step_num * 256)
        val_losses.append(val_loss_val)
    
    return {
        'lr': lr,
        'train_iters': train_iters,
        'train_losses': train_losses,
        'val_steps': val_steps,
        'val_losses': val_losses
    }

def thousands_formatter(x, pos):
    if x >= 1000:
        return f'{x/1000:.0f}k'
    return f'{x:.0f}'

def plot_loss_curves(*log_file_paths):
    # Parse all log files
    all_data = [parse_log_file(path) for path in log_file_paths]
    
    # Colors for different learning rates
    colors = ['blue', 'red', 'green', 'purple', 'orange', 'brown', 'pink', 'gray']
    
    # Set larger font sizes
    plt.rcParams.update({
        'font.size': 16,
        'axes.titlesize': 20,
        'axes.labelsize': 18,
        'xtick.labelsize': 16,
        'ytick.labelsize': 16,
        'legend.fontsize': 16
    })
    
    # Plot training losses
    plt.figure(figsize=(12, 6))
    for i, data in enumerate(all_data):
        color = colors[i % len(colors)]
        plt.plot(data['train_iters'], data['train_losses'], 
                 label=f'LR={data["lr"]}', color=color, linewidth=2.5)
    
    plt.xlabel('# Keys', fontsize=18, fontweight='bold')
    plt.ylabel('Train Loss', fontsize=18, fontweight='bold')
    plt.title('Training Loss Curves', fontsize=20, fontweight='bold')
    plt.grid(True)
    plt.legend(fontsize=16, frameon=True, facecolor='white', edgecolor='black')
    plt.gca().xaxis.set_major_formatter(FuncFormatter(thousands_formatter))
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.tight_layout()
    plt.show()

    # Plot validation losses
    plt.figure(figsize=(12, 6))
    for i, data in enumerate(all_data):
        if data['val_steps']:  # Only plot if validation data exists
            color = colors[i % len(colors)]
            plt.plot(data['val_steps'], data['val_losses'], 
                     label=f'LR={data["lr"]}', color=color, marker='o', markersize=10, linewidth=2.5)
    
    plt.xlabel('# Keys', fontsize=18, fontweight='bold')
    plt.ylabel('Val Loss', fontsize=18, fontweight='bold')
    plt.title('Validation Loss Curves', fontsize=20, fontweight='bold')
    plt.grid(True)
    plt.legend(fontsize=16, frameon=True, facecolor='white', edgecolor='black')
    plt.gca().xaxis.set_major_formatter(FuncFormatter(thousands_formatter))
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_loss_curves('logs/lr-1e-3-otp.log')

In [ ]:
plot_loss_curves('logs/lr-1e-3-64.log', 'logs/lr-1e-3-otp.log')

In [ ]:
@torch.no_grad()
def model_infer(input_seq, return_all=False):
    """
    Runs inference on a single input sequence and returns the model's output sequence.

    Args:
        input_seq (str): The input sequence (e.g., ciphertext or prompt).
        return_all (bool, optional): If True, returns all predicted tokens. 
                                    If False, returns only the last token. Defaults to False.

    Returns:
        If return_all=False:
            str: The predicted output (last token).
        If return_all=True:
            str: The complete predicted output sequence.
    """
    model.eval()

    # Convert the input sequence to tensor
    input_indices = torch.tensor([stoi[c] for c in input_seq], dtype=torch.long).unsqueeze(0).to(device)

    # Create a dummy Y for conditioning if required (same shape as input)
    dummy_Y = torch.zeros_like(input_indices).to(device)

    # Forward pass
    logits, _ = model(input_indices, dummy_Y)

    # Take the most likely token at each position
    predicted_indices = logits.argmax(dim=-1).squeeze(0)  # shape: (sequence length,)

    if return_all:
        # Convert back to string - full sequence
        output_seq = ''.join([itos[idx.item()] for idx in predicted_indices])
        return output_seq
    else:
        # Return only the last predicted token
        last_token = itos[predicted_indices[-1].item()]
        return last_token

def mono_alph_naive_decrypt(input_seq, return_all=True):
    d = {}
    output = []
    for i in range(len(input_seq)):
        if i % 2 == 0: # ciphertext
            c = input_seq[i]
            output.append(c)
            if c in d:
                output.append(d[c])
            else:
                output.append(' ')
        else: # plaintext
            m = input_seq[i]
            d[input_seq[i - 1]] = m
    return "".join(output[1:])

def mono_alph_freq_decrypt(input_seq, return_all=True):
    d = {}
    output = []
    known_plain_chars = set()
    freq_order = "etaoinshrdlcumwfgypbvkjxqz"
    freq_idx = 0
    for i in range(len(input_seq)):
        if i % 2 == 0: # ciphertext
            c = input_seq[i]
            output.append(c)
            if c in d:
                output.append(d[c])
            else:
                while freq_order[freq_idx] in known_plain_chars:
                    freq_idx += 1
                output.append(freq_order[freq_idx])
        else: # plaintext
            m = input_seq[i]
            d[input_seq[i - 1]] = m
            known_plain_chars.add(m)
    return "".join(output[1:])

In [ ]:
alpha_ids = np.array([stoi[c] for c in ALPHABET], dtype=np.uint8)
class EncryptionScheme():
    def enc(self, plain):
        raise NotImplementedError("Must be implemented in subclass")
    
    def dec(self, cipher):
        raise NotImplementedError("Must be implemented in subclass")

class MonoAlphabetic(EncryptionScheme):
    def __init__(self):
        self.perm = np.random.permutation(26)
        self.enc_map  = {alpha_ids[i]: alpha_ids[self.perm[i]] for i in range(26)}   # plain→cipher
        self.dec_map  = {v: k for k, v in self.enc_map.items()}                          # cipher→plain

    def enc(self, plain):
        return self.enc_map[plain]

    def dec(self, cipher):
        return self.dec_map[cipher]

class Vigenere(EncryptionScheme):
    def __init__(self, key_length):
        self.key_length = key_length
        self.key = np.random.choice(alpha_ids, size=key_length, replace=True)
        self.enc_idx = 0
        self.dec_idx = 0

    def enc(self, plain):
        shift = self.key[self.enc_idx % self.key_length]
        c = (plain + shift) % len(alpha_ids) 
        self.enc_idx += 1
        return c

    def dec(self, cipher):
        shift = self.key[self.dec_idx % self.key_length]
        p = (cipher - shift) % len(alpha_ids)
        self.dec_idx += 1
        return p

In [ ]:
# Sample a random sentence from the validation set
data_dir = os.path.join('data', dataset)
val_mmap = np.memmap(os.path.join(data_dir, 'val.bin'), dtype=np.uint8, mode='r')

# Get a random segment of text (let's say 30 characters)
sample_length = 100
start_idx = np.random.randint(0, len(val_mmap) - sample_length)
plain_ids = val_mmap[start_idx:start_idx + sample_length].copy()

# Convert to plaintext
plain = ''.join([itos[idx] for idx in plain_ids])
print(f"Plaintext: {plain}")

# Generate a random encryption key
scheme = MonoAlphabetic()
enc, dec = scheme.enc, scheme.dec

# Encrypt the plaintext
cipher_ids = np.array([enc(p) for p in plain_ids], dtype=np.uint8)
cipher = ''.join([itos[idx] for idx in cipher_ids])
print(f"Ciphertext: {cipher}")

# Create the prompt for the model (all but last character pairs)
known_k = len(plain_ids) - 1
buf = []
for i in range(known_k):
    buf.extend([cipher_ids[i], plain_ids[i]])
buf.append(cipher_ids[known_k])  # Add the last cipher character as query

# Convert to string for visualization
prompt_str = ''.join([itos[idx] for idx in buf])
print(f"Model prompt: {prompt_str}")

# Prepare input for model_infer
input_seq = prompt_str

# Get the ground truth (the last plaintext character)
ground_truth = itos[plain_ids[known_k]]
print(f"Ground truth (last character): {ground_truth}")

# The model will predict this in the next cell
print(f"The model will predict the decryption of '{itos[cipher_ids[known_k]]}' → ?")

In [ ]:
output = model_infer(input_seq, return_all=True)
output1 = mono_alph_naive_decrypt(input_seq, return_all = True)
print("out seq:", output[0:-1:2] + output[-1]) 
print("tru seq:", plain)
print(f"decrypting {itos[cipher_ids[known_k]]} →", output[-1])

print('---------------------------')
print("out seq:", output1[0:-1:2] + output1[-1]) 
print("tru seq:", plain)
print(f"decrypting {itos[cipher_ids[known_k]]} →", output1[-1])

In [ ]:
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np

plt.style.use('seaborn-v0_8-whitegrid')

def evaluate_in_context_learning(model_fn, scheme_name='mono', vig_key_size=None, num_samples=1000, max_examples=1023, uniform_sampling=False):
    position_correct = {}
    position_total = {}

    for _ in tqdm(range(num_samples), desc=f"Evaluating ({model_fn.__name__})"):
        sample_length = max_examples + 1
        if uniform_sampling:
            plain_ids = np.random.randint(0, len(alpha_ids), size=sample_length, dtype=np.uint8)
        else:
            start_idx = np.random.randint(0, len(val_mmap) - sample_length)
            plain_ids = val_mmap[start_idx:start_idx + sample_length].copy()
        
        if scheme_name == 'mono':
            scheme = MonoAlphabetic()
        elif scheme_name == 'vigenere':
            if vig_key_size is None:
                raise ValueError("poly_key_size must be specified for poly-alphabetic schemes")   
            if vig_key_size == 'random':
                scheme = Vigenere(np.random.randint(4, 32))
            else:
                scheme = Vigenere(vig_key_size)
        enc, dec = scheme.enc, scheme.dec
        cipher_ids = np.array([enc(p) for p in plain_ids], dtype=np.uint8)

        buf = []
        for i in range(max_examples):
            buf.extend([cipher_ids[i], plain_ids[i]])
        buf.append(cipher_ids[max_examples])

        prompt_str = ''.join([itos[idx] for idx in buf])
        output = model_fn(prompt_str, return_all=True)

        for i in range(max_examples):
            position = i + 1
            pred_idx = 2 * i
            ground_truth = itos[plain_ids[i]]
            pred_char = output[pred_idx]

            if position not in position_correct:
                position_correct[position] = 0
                position_total[position] = 0

            if pred_char == ground_truth:
                position_correct[position] += 1
            position_total[position] += 1

    positions = sorted(position_total.keys())
    accuracies = [position_correct[pos] / position_total[pos] for pos in positions]
    return positions, accuracies

def plot_accuracy_comparison(positions, *model_results, max_position=None):
    # Apply cutoff if max_position is set
    if max_position is not None:
        positions = [p for p in positions if p <= max_position]
        model_results = [
            (name, acc[:len(positions)])
            for name, acc in model_results
        ]

    for scale in ['log', 'linear']:
        plt.figure(figsize=(12, 6), dpi=100)

        for name, accuracies in model_results:
            plt.plot(positions, accuracies, label=name, linewidth=2.5, alpha=0.85)

        if scale == 'log':
            plt.xscale('log')
            xtick_vals = [p for p in [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1023, 1535] if p <= positions[-1]]
            plt.xticks(xtick_vals, [str(p) for p in xtick_vals], rotation=45)

        plt.xlabel('Number of In-Context Examples Seen', fontsize=12, fontweight='bold')
        plt.ylabel('Accuracy', fontsize=12, fontweight='bold')
        plt.title(f'Model Accuracy vs. Number of In-Context Examples ({scale.capitalize()} Scale)',
                  fontsize=14, fontweight='bold', pad=15)

        plt.grid(True, which="both", ls="--", alpha=0.3)
        plt.legend(fontsize=11)

        ax = plt.gca()
        ax.set_facecolor('#f8f9fa')
        for spine in ax.spines.values():
            spine.set_visible(True)
            spine.set_color('#cccccc')
            spine.set_linewidth(0.8)

        plt.tight_layout()
        plt.show()

# Mono-Alphabetic

## Training Distribution (OpenWebText)

In [ ]:
positions_mono_train_transformer, acc_mono_train_transformer = evaluate_in_context_learning(model_infer)

In [ ]:
positions_mono_train_naive, acc_mono_train_naive = evaluate_in_context_learning(mono_alph_naive_decrypt)

In [ ]:
positions_mono_train_freq, acc_mono_train_freq = evaluate_in_context_learning(mono_alph_freq_decrypt)

In [ ]:
plot_accuracy_comparison(
    positions_mono_train_transformer,
    ("Transformer", acc_mono_train_transformer),
    ("Naive Attack", acc_mono_train_naive),
    ("Frequency Based Attack", acc_mono_train_freq),
    max_position=128
)

## Uniform Distribution

In [ ]:
positions_mono_uniform_transformer, acc_mono_uniform_transformer = evaluate_in_context_learning(model_infer, uniform_sampling=True)

In [ ]:
positions_mono_uniform_naive, acc_mono_uniform_naive = evaluate_in_context_learning(mono_alph_naive_decrypt, uniform_sampling=True)

In [ ]:
positions_mono_uniform_freq, acc_mono_uniform_freq = evaluate_in_context_learning(mono_alph_freq_decrypt, uniform_sampling=True)

In [ ]:
plot_accuracy_comparison(
    positions_mono_uniform_transformer,
    ("Transformer", acc_mono_uniform_transformer),
    ("Naive Attack", acc_mono_uniform_naive),
    ("Frequency Based Attack", acc_mono_uniform_freq),
    max_position=128
)

# Vigenere

In [ ]:
def ch2i(ch: str):
    return ord(ch) - 97 
    
def i2ch(i: int):        
    return chr((i % 26) + 97)

def vigenere_naive_decrypt(key_len):

    def func(input_seq, return_all=True):
        
        key_table = [None] * key_len    
        output    = []
        
        for i, ch in enumerate(input_seq):
            if i % 2 == 0:                       
                c = ch
                output.append(c)                
                
                pos = i // 2              
                key_idx = pos % key_len
                key_offset = key_table[key_idx]
                
                if key_offset is not None:      
                    p = i2ch((ch2i(c) - key_offset) % 26)
                else:                            
                    p = ' '
                output.append(p)
            
            else:                               
                p = ch
                c = input_seq[i - 1]
                pos     = (i - 1) // 2
                key_idx = pos % key_len
                key_table[key_idx] = (ch2i(c) - ch2i(p)) % 26 
        
        decoded_seq = "".join(output[1:])        
        
        return decoded_seq if return_all else (decoded_seq[-1] if decoded_seq else "")
        
    return func

def vigenere_freq_decrypt(key_len):

    def func(input_seq, return_all=True):
        
        key_table = [None] * key_len    
        output    = []
        
        for i, ch in enumerate(input_seq):
            if i % 2 == 0:                       
                c = ch
                output.append(c)                
                
                pos = i // 2              
                key_idx = pos % key_len
                key_offset = key_table[key_idx]
                
                if key_offset is not None:      
                    p = i2ch((ch2i(c) - key_offset) % 26)
                else:                            
                    p = 'e'
                output.append(p)
            
            else:                               
                p = ch
                c = input_seq[i - 1]
                pos     = (i - 1) // 2
                key_idx = pos % key_len
                key_table[key_idx] = (ch2i(c) - ch2i(p)) % 26 
        
        decoded_seq = "".join(output[1:])        
        
        return decoded_seq if return_all else (decoded_seq[-1] if decoded_seq else "")
        
    return func

## Key Size: 4

### Training Distribution

In [ ]:
positions_vig_train_transformer, acc_vig_train_transformer = evaluate_in_context_learning(model_infer, scheme_name='vigenere', vig_key_size=4)

In [ ]:
positions_vig_train_naive, acc_vig_train_naive = evaluate_in_context_learning(vigenere_naive_decrypt(4), scheme_name='vigenere', vig_key_size=4)

In [ ]:
positions_vig_train_freq, acc_vig_train_freq = evaluate_in_context_learning(vigenere_freq_decrypt(4), scheme_name='vigenere', vig_key_size=4)

In [ ]:
plot_accuracy_comparison(
    positions_vig_train_transformer,
    ("Transformer", acc_vig_train_transformer),
    ("Naive Attack", acc_vig_train_naive),
    ("Frequency Based Attack", acc_vig_train_freq),
    max_position=128
)

### Uniform Distribution

In [ ]:
positions_vig_uniform_transformer, acc_vig_uniform_transformer = evaluate_in_context_learning(model_infer, scheme_name='vigenere', vig_key_size=4, uniform_sampling=True)

In [ ]:
positions_vig_uniform_naive, acc_vig_uniform_naive = evaluate_in_context_learning(vigenere_naive_decrypt(4), scheme_name='vigenere', vig_key_size=4, uniform_sampling=True)

In [ ]:
positions_vig_uniform_freq, acc_vig_uniform_freq = evaluate_in_context_learning(vigenere_freq_decrypt(4), scheme_name='vigenere', vig_key_size=4, uniform_sampling=True)

In [ ]:
plot_accuracy_comparison(
    positions_vig_train_transformer,
    ("Transformer", acc_vig_uniform_transformer),
    ("Naive Attack", acc_vig_uniform_naive),
    ("Frequency Based Attack", acc_vig_uniform_freq),
    max_position=None
)

## Key Size: 32

### Training Distribution

In [ ]:
positions_vig_train_transformer, acc_vig_train_transformer = evaluate_in_context_learning(model_infer, scheme_name='vigenere', vig_key_size=32)

In [ ]:
positions_vig_train_naive, acc_vig_train_naive = evaluate_in_context_learning(vigenere_naive_decrypt(32), scheme_name='vigenere', vig_key_size=32)

In [ ]:
positions_vig_train_freq, acc_vig_train_freq = evaluate_in_context_learning(vigenere_freq_decrypt(32), scheme_name='vigenere', vig_key_size=32)

In [ ]:
plot_accuracy_comparison(
    positions_vig_train_transformer,
    ("Transformer", acc_vig_train_transformer),
    ("Naive Attack", acc_vig_train_naive),
    ("Frequency Based Attack", acc_vig_train_freq),
    max_position=128
)

### Uniform Distribution

In [ ]:
positions_vig_uniform_transformer, acc_vig_uniform_transformer = evaluate_in_context_learning(model_infer, scheme_name='vigenere', vig_key_size=32, uniform_sampling=True)

In [ ]:
positions_vig_uniform_naive, acc_vig_uniform_naive = evaluate_in_context_learning(vigenere_naive_decrypt(32), scheme_name='vigenere', vig_key_size=32, uniform_sampling=True)

In [ ]:
positions_vig_uniform_freq, acc_vig_uniform_freq = evaluate_in_context_learning(vigenere_freq_decrypt(32), scheme_name='vigenere', vig_key_size=32, uniform_sampling=True)

In [ ]:
plot_accuracy_comparison(
    positions_vig_train_transformer,
    ("Transformer", acc_vig_uniform_transformer),
    ("Naive Attack", acc_vig_uniform_naive),
    ("Frequency Based Attack", acc_vig_uniform_freq),
    max_position=None
)

### Train Distribution, Key Len = 16

In [ ]:
positions_vig_train_transformer, acc_vig_train_transformer = evaluate_in_context_learning(model_infer, scheme_name='vigenere', vig_key_size=16)

In [ ]:
positions_vig_train_freq_32, acc_vig_train_freq_32 = evaluate_in_context_learning(vigenere_freq_decrypt(32), scheme_name='vigenere', vig_key_size=16)

In [ ]:
positions_vig_train_freq_16, acc_vig_train_freq_16 = evaluate_in_context_learning(vigenere_freq_decrypt(16), scheme_name='vigenere', vig_key_size=16)

In [ ]:
plot_accuracy_comparison(
    positions_vig_train_transformer,
    ("Transformer", acc_vig_train_transformer),
    ("Frequency Based Attack on Key Length 32", acc_vig_train_freq_32),
    ("Frequency Based Attack on Key Length 16", acc_vig_train_freq_16),
    max_position=None
)

### Train Distribution, Key Len = 20

In [ ]:
positions_vig_train_transformer, acc_vig_train_transformer = evaluate_in_context_learning(model_infer, scheme_name='vigenere', vig_key_size=20)

In [ ]:
positions_vig_train_freq_32, acc_vig_train_freq_32 = evaluate_in_context_learning(vigenere_freq_decrypt(32), scheme_name='vigenere', vig_key_size=20)

In [ ]:
positions_vig_train_freq_20, acc_vig_train_freq_20 = evaluate_in_context_learning(vigenere_freq_decrypt(20), scheme_name='vigenere', vig_key_size=20)

In [ ]:
plot_accuracy_comparison(
    positions_vig_train_transformer,
    ("Transformer", acc_vig_train_transformer),
    ("Frequency Based Attack on Key Length 32", acc_vig_train_freq_32),
    ("Frequency Based Attack on Key Length 20", acc_vig_train_freq_20),
    max_position=None
)

## Key Length: Random (4 - 32)

In [ ]:
def vigenere_unknownlen_naive(input_seq: str, return_all: bool = True):
    min_len = 4
    max_len = 32
    candidates = [
        {"len": L, "table": [None] * L, "valid": True} for L in range(min_len, max_len + 1)
    ]
    output = []

    for i, ch in enumerate(input_seq):
        if i % 2 == 0:
            c = ch
            output.append(c)

            pos = i // 2
            shifts = []
            for cand in candidates:
                if not cand["valid"]:
                    continue
                shift = cand["table"][pos % cand["len"]]
                shifts.append(shift)

            known = [s for s in shifts if s is not None]
            if known and all(s == known[0] for s in known):
                p = i2ch((ch2i(c) - known[0]) % 26)
            else:
                p = " "          
            output.append(p)

        else:
            p = ch
            c = input_seq[i - 1]
            pos = (i - 1) // 2
            want = (ch2i(c) - ch2i(p)) % 26

            for cand in candidates:
                if not cand["valid"]:
                    continue
                kidx = pos % cand["len"]
                known = cand["table"][kidx]
                if known is None:
                    cand["table"][kidx] = want
                elif known != want:
                    cand["valid"] = False  

    decoded = "".join(output[1:])  
    return decoded if return_all else (decoded[-1] if decoded else "")

### Training Distribution

In [ ]:
positions_vig_train_transformer, acc_vig_train_transformer = evaluate_in_context_learning(model_infer, max_examples=1535, scheme_name='vigenere', vig_key_size='random')

In [ ]:
positions_vig_train_naive, acc_vig_train_naive = evaluate_in_context_learning(vigenere_unknownlen_naive, max_examples=1535, scheme_name='vigenere', vig_key_size='random')

In [ ]:
plot_accuracy_comparison(
    positions_vig_train_transformer,
    ("Transformer", acc_vig_train_transformer),
    ("Naive Attack", acc_vig_train_naive),
    max_position=None
)

### Uniform Distribution

In [ ]:
positions_vig_uniform_transformer, acc_vig_uniform_transformer = evaluate_in_context_learning(model_infer, max_examples=1535, scheme_name='vigenere', vig_key_size='random', uniform_sampling=True)

In [ ]:
positions_vig_uniform_naive, acc_vig_uniform_naive = evaluate_in_context_learning(vigenere_unknownlen_naive, max_examples=1535, scheme_name='vigenere', vig_key_size='random', uniform_sampling=True)

In [ ]:
plot_accuracy_comparison(
    positions_vig_uniform_transformer,
    ("Transformer", acc_vig_uniform_transformer),
    ("Naive Attack", acc_vig_uniform_naive),
    max_position=None
)

### Training Distribution, Key Len = 32

In [ ]:
positions_vig_train_transformer, acc_vig_train_transformer = evaluate_in_context_learning(model_infer, max_examples=1535, scheme_name='vigenere', vig_key_size=32)

In [ ]:
positions_vig_train_naive, acc_vig_train_naive = evaluate_in_context_learning(vigenere_unknownlen_naive, max_examples=1535, scheme_name='vigenere', vig_key_size=32)

In [ ]:
positions_vig_train_freq_32, acc_vig_train_freq_32 = evaluate_in_context_learning(vigenere_freq_decrypt(32), max_examples=1535, scheme_name='vigenere', vig_key_size=32)

In [ ]:
plot_accuracy_comparison(
    positions_vig_uniform_transformer,
    ("Transformer", acc_vig_train_transformer),
    ("Naive Attack on Unknown Key Length", acc_vig_train_naive),
    ("Frequency Based Attack on Key Length 32", acc_vig_train_freq_32),
    max_position=None
)